In [9]:
login_url = "https://www.linkedin.com/jobs/search/?currentJobId=0&f_E=1&f_TPR=r86400&keywords=data%20science%20praktikum&origin=JOB_SEARCH_PAGE_JOB_FILTER"

KeyboardInterrupt: 

In [11]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import ollama
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import re
import random
import json
import ollama
import os
from dotenv import load_dotenv
import webbrowser

In [17]:
def startEngine():
    #session = requests.session()
    #session.headers.update({
    #'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.125 Safari/537.36',
    #'Referer': 'https://google.com'
    #})
    load_dotenv()
    mail = os.getenv("mail_1")
    password = os.getenv("password_1")
    driver = webdriver.Chrome()
    driver.get(login_url)
    time.sleep(2)
    login_button = driver.find_element("xpath", '//*[@id="base-contextual-sign-in-modal"]/div/section/div/div/div/div[2]/button')
    login_button.click()
    time.sleep(2)
    e_mail = driver.find_element("xpath", '//*[@id="base-sign-in-modal_session_key"]')
    e_mail.send_keys(mail)
    password_input = driver.find_element("xpath", '//*[@id="base-sign-in-modal_session_password"]')
    password_input.send_keys(password)

    button_2 = driver.find_element("xpath", '//*[@id="base-sign-in-modal"]/div/section/div/div/form/div[2]/button')
    button_2.click()
    time.sleep(5)
    return driver

#driver = startEngine()

In [13]:
def get_Click_Directions(driver):
    jobs_all = driver.find_elements(By.CSS_SELECTOR, "li.scaffold-layout__list-item")
    #ember-view   URsVkPwtVvyhSWvdlbuQwZoPpGmMOdMCRjZmWE occludable-update p0 relative scaffold-layout__list-item
    

    return jobs_all

#jobs = get_Click_Directions(driver)


In [14]:
# get the infos

def Job_Content(driver):
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    About_The_Job = soup.find('article', class_='jobs-description__container') # all jobs
    Company_Name = soup.find('div', class_='job-details-jobs-unified-top-card__company-name')
    titel_h1 = soup.find('div', class_="job-details-jobs-unified-top-card__job-title")
    Link_to_stelle = titel_h1.find('a')

    return About_The_Job.get_text(strip =True), Company_Name.get_text(strip = True), Link_to_stelle["href"]

#job_description, Company_Name, Location, Link_to_stelle = Job_Content()


In [15]:
def Ollama_Rate(job_description):
    prompt = f"""
        Analyze the following job posting for an INTERNSHIP.

        CONTEXT:
        - Job Description: {job_description[:4000]}

        ==================================================
        TASK:
        Determine if this job is suitable for a Bachelor internship based on the rules below.

        ==================================================
        STEP 1 — CRITICAL FACT CHECK
        Check the text for the following constraints. Answer internally with YES/NO.

        1. IS_INTERNSHIP: Does it explicitly offer an Internship/Praktikum? (Reject if only Working Student/Werkstudent).
        2. DURATION_OK: Is the duration at least 5 months (or "3-6 months", "6 months")? (Reject if only 3 months or less).
        3. LOCATION_OK: Is it NOT fully remote? (Accept hybrid/on-site).
        4. LEVEL_OK: Is it suitable for Bachelor students?
        5. FIELD_OK: Is it in a relevant field (e.g., Data Science, Software Engineering)?

        ==================================================
        DECISION LOGIC:
        - If ANY strict negative rule applies (e.g., only Werkstudent, too short) -> REJECT.
        - If specific duration is missing but context implies a substantial internship -> ACCEPT.
        
        ==================================================
        OUTPUT FORMAT:
        Return ONLY a valid JSON object. Do not include markdown formatting (like ```json).

        {{
        "status": "accepted" OR "rejected",
        "reason": "Brief explanation why (max 10 words). E.g., 'Only 3 months duration' or 'Only Werkstudent offered'"
        }}
        """
    ollama_filter = ollama.chat(
        model="qwen3:8b", 
        format="json",
        messages=[{'role': 'user', 'content': prompt}],
        options={"temperature": 0.0}
        )
    chat = ollama_filter["message"]['content']
    response = json.loads(chat)
    return response

In [20]:
driver = startEngine()



In [21]:

job_container = []
regex_logic = r"\/jobs\/view\/\d+"
counter = 0
while True:


    jobs = get_Click_Directions(driver) # where to click: Job Listings path
    print(f"Found {len(jobs)} jobs on this page.")
    
    time.sleep(5)

    for job in jobs:
        x = random.uniform(1.0, 3.0)
        y = random.uniform(1.5, 3.5)
        #time.sleep(x)
        job.click()
        time.sleep(y)
        print("is being processed", job)
        job_description, Company_Name, Link_to_stelle = Job_Content(driver)
        print("ollama thinks")
        ollama_response = Ollama_Rate(job_description)
        print("ollama said", ollama_response)
        
        if ollama_response.get("status") == "accepted":
            job_container.append([
                Job_Content(driver)
                ])
            #if job_container[counter][1] >= 55:  # rating filetr
            print("Praktikum Gefunden")    # open the link in a new tab, if the rate is > 100
            print("link",job_container[counter])
            match = re.search(regex_logic, job_container[counter][0][2])
            print(match)
            find_the_link = match.group()
            print("new Tab")
            create_link = f"https://www.linkedin.com{find_the_link}"
            print(create_link)
            #webbrowser.open_new_tab(create_link)
            cmd = f"open -g {create_link}"
            os.system(cmd)
            counter += 1


        else:
            continue

    class_name = 'artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--tertiary ember-view jobs-search-pagination__button jobs-search-pagination__button--next'.replace(" ", ".")
    print(f"No more pages ")
    try:
        next_page_button = driver.find_element(By.CLASS_NAME, f'{class_name}')
        next_page_button.click()
    except Exception as e:
        print(f"No more pages to click. Exiting. {e}")
        break




Found 25 jobs on this page.
is being processed <selenium.webdriver.remote.webelement.WebElement (session="31ec05c4d2d2d31ffc3b68f5b6bf7cc8", element="f.B677E31CF3FFFB63BE8A4BCFC4A9BE2C.d.FD4C88B4493FA229B096BC0BE414C351.e.133")>
ollama thinks
ollama said {'status': 'rejected', 'reason': 'Only Werkstudent offered'}
is being processed <selenium.webdriver.remote.webelement.WebElement (session="31ec05c4d2d2d31ffc3b68f5b6bf7cc8", element="f.B677E31CF3FFFB63BE8A4BCFC4A9BE2C.d.FD4C88B4493FA229B096BC0BE414C351.e.134")>
ollama thinks
ollama said {'status': 'accepted', 'reason': 'Suitable for Bachelor level with relevant field'}
Praktikum Gefunden
link [('About the jobProject descriptionThe aim of this project is to support the Tier-1 in development and maintenance of XIL User Suite and Hil sensor injection.Injection engineering should be supported for:• Radar• Global navigation & satellite system• Ultrasonic• Driver Monitoring System• CameraThe XIL framework is to support the testing of System 

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=143.0.7499.42)
Stacktrace:
0   chromedriver                        0x0000000104d26d7c cxxbridge1$str$ptr + 3028012
1   chromedriver                        0x0000000104d1ec3c cxxbridge1$str$ptr + 2994924
2   chromedriver                        0x000000010481ab1c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74196
3   chromedriver                        0x00000001048042d4 chromedriver + 213716
4   chromedriver                        0x00000001048275c4 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 126076
5   chromedriver                        0x0000000104889c50 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 529160
6   chromedriver                        0x00000001048a24dc _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 629652

In [26]:
len(job_container)

1

In [27]:
job_container[0][0][2]

'/jobs/view/4350083389/?alternateChannel=search&eBP=NOT_ELIGIBLE_FOR_CHARGING&refId=ycTaYDSxZ9J3BEqFa%2F%2FbYQ%3D%3D&trackingId=g8X%2B%2BVbYj524ARJsKDwwJw%3D%3D&trk=d_flagship3_search_srp_jobs'